<a href="https://colab.research.google.com/github/brahma1995/Bird_species_classification/blob/pytorch_child/pytorch_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/mrdbourke/pytorch-deep-learning/blob/main/06_pytorch_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[View Source Code](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/06_pytorch_transfer_learning.ipynb) | [View Slides](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/slides/06_pytorch_transfer_learning.pdf)

In [ ]:
! pip install Kaggle
!  mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! kaggle datasets download -d kedarsai/bird-species-classification-220-categories
!unzip bird-species-classification-220-categories.zip

Streaming output truncated to the last 5000 lines.
  inflating: Train/Horned_Puffin/Horned_Puffin_0029_100888.jpg  
  inflating: Train/Horned_Puffin/Horned_Puffin_0030_100725.jpg  
  inflating: Train/Horned_Puffin/Horned_Puffin_0031_100804.jpg  
  inflating: Train/Horned_Puffin/Horned_Puffin_0033_100731.jpg  
  inflating: Train/Horned_Puffin/Horned_Puffin_0034_100895.jpg  
  inflating: Train/Horned_Puffin/Horned_Puffin_0035_100690.jpg  
  inflating: Train/Horned_Puffin/Horned_Puffin_0036_100974.jpg  
  inflating: Train/Horned_Puffin/Horned_Puffin_0038_100635.jpg  
  inflating: Train/Horned_Puffin/Horned_Puffin_0039_100890.jpg  
  inflating: Train/Horned_Puffin/Horned_Puffin_0040_100891.jpg  
  inflating: Train/Horned_Puffin/Horned_Puffin_0042_100760.jpg  
  inflating: Train/Horned_Puffin/Horned_Puffin_0045_100803.jpg  
  inflating: Train/Horned_Puffin/Horned_Puffin_0046_100785.jpg  
  inflating: Train/Horned_Puffin/Horned_Puffin_0047_100967.jpg  
  inflating: Train/Horned_Puffin/Horned

In [ ]:
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

[INFO] torch/torchvision versions not as required, installing nightly versions.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
torch version: 2.0.1+cu118
torchvision version: 0.15.2+cu118


In [ ]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms,datasets
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from typing import Dict, List, Tuple
import os

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary



[INFO] Couldn't find torchinfo... installing it.


In [ ]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
# from google.colab import files
# files.download('/content/drive/MyDrive/bird_dataset_200.zip')

In [ ]:
# Setup Dirs
train_dir = "Train"
test_dir = "Test"

In [ ]:
# Get a set of pretrained model weights


weights = torchvision.models.EfficientNet_B5_Weights.DEFAULT # .DEFAULT = best available weights from pretraining on ImageNet
weights

EfficientNet_B5_Weights.IMAGENET1K_V1

In [ ]:
# Get the transforms used to create our pretrained weights
auto_transforms = weights.transforms()
auto_transforms

ImageClassification(
    crop_size=[456]
    resize_size=[456]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train_dataloader':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'test_dataloader':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {
    'train_dataloader':
    datasets.ImageFolder(train_dir, data_transforms['train_dataloader']),
    'test_dataloader':
    datasets.ImageFolder(test_dir, data_transforms['test_dataloader'])
}

dataloaders = {
    'train_dataloader':
    torch.utils.data.DataLoader(image_datasets['train_dataloader'],
                                batch_size=32,
                                shuffle=True,
                                num_workers=0),  # for Kaggle
    'test_dataloader':
    torch.utils.data.DataLoader(image_datasets['test_dataloader'],
                                batch_size=32,
                                shuffle=False,
                                num_workers=0)  # for Kaggle
}
# Get class names
class_names = image_datasets['train_dataloader'].classes

In [ ]:
train_dataloader = dataloaders['train_dataloader']
test_dataloader = dataloaders['test_dataloader']
len(train_dataloader), len(test_dataloader), len(class_names)

(295, 75, 200)

In [ ]:
# OLD: Setup the model with pretrained weights and send it to the target device (this was prior to torchvision v0.13)
# model = torchvision.models.efficientnet_b0(pretrained=True).to(device) # OLD method (with pretrained=True)

# NEW: Setup the model with pretrained weights and send it to the target device (torchvision v0.13+)
weights = torchvision.models.EfficientNet_B5_Weights.DEFAULT.DEFAULT # .DEFAULT = best available weights
model = torchvision.models.efficientnet_b5(weights=weights).to(device)

#model # uncomment to output (it's very long)

Downloading: "https://download.pytorch.org/models/efficientnet_b5_lukemelas-b6417697.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b5_lukemelas-b6417697.pth
100%|██████████| 117M/117M [00:00<00:00, 286MB/s]


In [ ]:
# Print a summary using torchinfo (uncomment for actual output)
summary(model=model,
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        #col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   True
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 2048, 7, 7]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 48, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 48, 112, 112]   1,296                True
│    │    └─BatchNorm2d (1)                                  [32, 48, 112, 112]   [32, 48, 112, 112]   96                   True
│    │    └─SiLU (2)                                         [32, 48, 112, 112]   [32, 48, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 48, 112, 112]   [32, 24, 112

In [ ]:
# Freeze all base layers in the "features" section of the model (the feature extractor) by setting requires_grad=False
# works well for vgg
# for param in model.features.parameters():
#     param.requires_grad = False

# for resenet because resnet has no features function
for param in model.parameters():
    param.requires_grad = False

In [ ]:
# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Get the length of class_names (one output unit for each class)
output_shape = len(class_names)

# # Recreate the classifier layer and seed it to the target device
# model.classifier = torch.nn.Sequential(
#     torch.nn.Dropout(p=0.2, inplace=True),
#     torch.nn.Linear(in_features=1280,
#                     out_features=output_shape, # same number of output units as our number of classes
#                     bias=True)).to(device)

# Change the final layer of ResNet50 Model for Transfer Learning
# fc_inputs = model.fc.in_features
# model.fc = nn.Sequential(
#     nn.Linear(fc_inputs, 256),
#     nn.ReLU(inplace=True),
#     nn.Dropout(0.4),
#     nn.Linear(256, output_shape)

# )
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=2048,
                    out_features=output_shape, # same number of output units as our number of classes
                    bias=True))



In [ ]:
# # Do a summary *after* freezing the features and changing the output classifier layer (uncomment for actual output)
summary(model,
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape" (batch_size, color_channels, height, width)
        verbose=0,
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 200]            --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 2048, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 48, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 48, 112, 112]   (1,296)              False
│    │    └─BatchNorm2d (1)                                  [32, 48, 112, 112]   [32, 48, 112, 112]   (96)                 False
│    │    └─SiLU (2)                                         [32, 48, 112, 112]   [32, 48, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 48, 112, 112]   [32, 

In [ ]:
# Define loss and optimizer
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=True)


NameError: ignored

In [ ]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               scheduler:torch.optim.lr_scheduler,
               device: torch.device) -> Tuple[float, float]:

    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # 6. scheduler step
        scheduler.step(loss)

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:

    # Put model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc



def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          scheduler:torch.optim.lr_scheduler,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:

    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Make sure model on target device
    model.to(device)

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          scheduler=scheduler,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)


        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        model_save_name = 'classifier.pt'
        path = F"/content/drive/MyDrive/bird_dataset_200/{model_save_name}"
        torch.save(model.state_dict(), path)

    # Return the filled results at the end of the epochs
    return results


In [ ]:
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Setup training and save the results
results = train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       scheduler = scheduler,
                       loss_fn=loss_fn,
                       epochs=30,
                       early_stop_callback=True,
                       device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 00006: reducing learning rate of group 0 to 5.0000e-04.
Epoch 00009: reducing learning rate of group 0 to 2.5000e-05.
Epoch 00012: reducing learning rate of group 0 to 1.2500e-06.
Epoch 00018: reducing learning rate of group 0 to 6.2500e-08.
Epoch 00021: reducing learning rate of group 0 to 3.1250e-09.
Epoch: 1 | train_loss: 5.3049 | train_acc: 0.0045 | test_loss: 5.2955 | test_acc: 0.0066


100%
50/50 [2:09:19<00:00, 155.08s/it]
Epoch: 1 | train_loss: 4.7887 | train_acc: 0.1451 | test_loss: 4.1300 | test_acc: 0.3171
Epoch: 2 | train_loss: 3.8100 | train_acc: 0.3510 | test_loss: 3.5354 | test_acc: 0.3621
Epoch: 3 | train_loss: 3.2195 | train_acc: 0.4253 | test_loss: 3.1267 | test_acc: 0.3933
Epoch: 4 | train_loss: 2.8269 | train_acc: 0.4756 | test_loss: 2.8602 | test_acc: 0.4314
Epoch: 5 | train_loss: 2.5601 | train_acc: 0.5067 | test_loss: 2.6962 | test_acc: 0.4414
Epoch: 6 | train_loss: 2.3464 | train_acc: 0.5362 | test_loss: 2.5604 | test_acc: 0.4524
Epoch: 7 | train_loss: 2.2072 | train_acc: 0.5471 | test_loss: 2.4675 | test_acc: 0.4611
Epoch: 8 | train_loss: 2.1002 | train_acc: 0.5610 | test_loss: 2.3987 | test_acc: 0.4615
Epoch: 9 | train_loss: 1.9585 | train_acc: 0.5937 | test_loss: 2.3292 | test_acc: 0.4724
Epoch: 10 | train_loss: 1.8878 | train_acc: 0.5986 | test_loss: 2.3006 | test_acc: 0.4761
Epoch: 11 | train_loss: 1.8131 | train_acc: 0.6095 | test_loss: 2.2566 | test_acc: 0.4811
Epoch: 12 | train_loss: 1.7595 | train_acc: 0.6165 | test_loss: 2.2263 | test_acc: 0.4744
Epoch: 13 | train_loss: 1.7005 | train_acc: 0.6245 | test_loss: 2.2048 | test_acc: 0.4811
Epoch: 14 | train_loss: 1.6325 | train_acc: 0.6391 | test_loss: 2.1776 | test_acc: 0.4890
Epoch: 15 | train_loss: 1.5837 | train_acc: 0.6541 | test_loss: 2.1621 | test_acc: 0.4928
Epoch: 16 | train_loss: 1.5415 | train_acc: 0.6518 | test_loss: 2.1503 | test_acc: 0.4924
Epoch: 17 | train_loss: 1.5008 | train_acc: 0.6561 | test_loss: 2.1396 | test_acc: 0.4874
Epoch: 18 | train_loss: 1.4894 | train_acc: 0.6669 | test_loss: 2.1251 | test_acc: 0.4869
Epoch: 19 | train_loss: 1.4380 | train_acc: 0.6758 | test_loss: 2.1145 | test_acc: 0.4890
Epoch: 20 | train_loss: 1.4207 | train_acc: 0.6691 | test_loss: 2.0995 | test_acc: 0.4889
Epoch: 21 | train_loss: 1.3703 | train_acc: 0.6843 | test_loss: 2.0868 | test_acc: 0.4906
Epoch: 22 | train_loss: 1.3796 | train_acc: 0.6787 | test_loss: 2.0820 | test_acc: 0.4856
Epoch: 23 | train_loss: 1.3209 | train_acc: 0.6947 | test_loss: 2.0804 | test_acc: 0.4910
Epoch: 24 | train_loss: 1.2935 | train_acc: 0.7010 | test_loss: 2.0786 | test_acc: 0.4979
Epoch: 25 | train_loss: 1.2948 | train_acc: 0.6992 | test_loss: 2.0724 | test_acc: 0.4939
Epoch: 26 | train_loss: 1.2680 | train_acc: 0.7026 | test_loss: 2.0631 | test_acc: 0.4982
Epoch: 27 | train_loss: 1.2552 | train_acc: 0.7029 | test_loss: 2.0609 | test_acc: 0.4947
Epoch: 28 | train_loss: 1.2265 | train_acc: 0.7101 | test_loss: 2.0636 | test_acc: 0.4978
Epoch: 29 | train_loss: 1.2148 | train_acc: 0.7133 | test_loss: 2.0672 | test_acc: 0.4956
Epoch: 30 | train_loss: 1.1779 | train_acc: 0.7209 | test_loss: 2.0600 | test_acc: 0.4983
Epoch: 31 | train_loss: 1.1815 | train_acc: 0.7209 | test_loss: 2.0596 | test_acc: 0.4940
Epoch: 32 | train_loss: 1.1508 | train_acc: 0.7257 | test_loss: 2.0591 | test_acc: 0.4919
Epoch: 33 | train_loss: 1.1498 | train_acc: 0.7217 | test_loss: 2.0625 | test_acc: 0.4950
Epoch: 34 | train_loss: 1.1364 | train_acc: 0.7286 | test_loss: 2.0663 | test_acc: 0.4940
Epoch: 35 | train_loss: 1.1264 | train_acc: 0.7215 | test_loss: 2.0639 | test_acc: 0.4965
Epoch: 36 | train_loss: 1.1228 | train_acc: 0.7286 | test_loss: 2.0671 | test_acc: 0.4919
Epoch: 37 | train_loss: 1.0811 | train_acc: 0.7374 | test_loss: 2.0653 | test_acc: 0.4957
Epoch: 38 | train_loss: 1.0883 | train_acc: 0.7365 | test_loss: 2.0623 | test_acc: 0.4940
Epoch: 39 | train_loss: 1.0674 | train_acc: 0.7370 | test_loss: 2.0619 | test_acc: 0.4954
Epoch: 40 | train_loss: 1.0478 | train_acc: 0.7466 | test_loss: 2.0658 | test_acc: 0.4946
Epoch: 41 | train_loss: 1.0665 | train_acc: 0.7417 | test_loss: 2.0692 | test_acc: 0.4897
Epoch: 42 | train_loss: 1.0356 | train_acc: 0.7445 | test_loss: 2.0728 | test_acc: 0.4881
Epoch: 43 | train_loss: 1.0121 | train_acc: 0.7558 | test_loss: 2.0596 | test_acc: 0.4901
Epoch: 44 | train_loss: 1.0139 | train_acc: 0.7517 | test_loss: 2.0624 | test_acc: 0.4924
Epoch: 45 | train_loss: 1.0032 | train_acc: 0.7528 | test_loss: 2.0589 | test_acc: 0.4931
Epoch: 46 | train_loss: 1.0030 | train_acc: 0.7525 | test_loss: 2.0626 | test_acc: 0.4956
Epoch: 47 | train_loss: 0.9830 | train_acc: 0.7556 | test_loss: 2.0641 | test_acc: 0.4944
Epoch: 48 | train_loss: 0.9638 | train_acc: 0.7654 | test_loss: 2.0518 | test_acc: 0.4928
Epoch: 49 | train_loss: 0.9529 | train_acc: 0.7629 | test_loss: 2.0613 | test_acc: 0.4953
Epoch: 50 | train_loss: 0.9473 | train_acc: 0.7639 | test_loss: 2.0601 | test_acc: 0.4965
# [INFO] Total training time: 7759.866 seconds

In [ ]:

# Plot the loss curves of our model
def plot_loss_curves(results):

    loss = results["train_loss"]
    test_loss = results["test_loss"]

    accuracy = results["train_acc"]
    test_accuracy = results["test_acc"]

    epochs = range(len(results["train_loss"]))

    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label="train_loss")
    plt.plot(epochs, test_loss, label="test_loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label="train_accuracy")
    plt.plot(epochs, test_accuracy, label="test_accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.legend()

plot_loss_curves(results)

In [ ]:
from typing import List, Tuple

from PIL import Image

# 1. Take in a trained model, class names, image path, image size, a transform and target device
def pred_and_plot_image(model: torch.nn.Module,
                        image_path: str,
                        class_names: List[str],
                        image_size: Tuple[int, int] = (224, 224),
                        transform: torchvision.transforms = None,
                        device: torch.device=device):


    # 2. Open image
    img = Image.open(image_path)

    # 3. Create transformation for image (if one doesn't exist)
    if transform is not None:
        image_transform = transform
    else:
        image_transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])

    ### Predict on image ###

    # 4. Make sure the model is on the target device
    model.to(device)

    # 5. Turn on model evaluation mode and inference mode
    model.eval()
    with torch.inference_mode():
      # 6. Transform and add an extra dimension to image (model requires samples in [batch_size, color_channels, height, width])
      transformed_image = image_transform(img).unsqueeze(dim=0)

      # 7. Make a prediction on image with an extra dimension and send it to the target device
      target_image_pred = model(transformed_image.to(device))

    # 8. Convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)

    # 9. Convert prediction probabilities -> prediction labels
    target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)

    # 10. Plot image with predicted label and probability
    plt.figure()
    plt.imshow(img)
    plt.title(f"Pred: {class_names[target_image_pred_label]} | Prob: {target_image_pred_probs.max():.3f}")
    plt.axis(False);

In [ ]:
# Get a random list of image paths from test set
import random,os
from pathlib import Path
num_images_to_plot = 3

test_image_path_list = list(Path(test_dir).glob("*/*.jpg")) # get list all image paths from test data
test_image_path_sample = random.sample(population=test_image_path_list, # go through all of the test image paths
                                       k=num_images_to_plot) # randomly select 'k' image paths to pred and plot

# Make predictions on and plot the images
for image_path in test_image_path_sample:
    pred_and_plot_image(model=model,
                        image_path=image_path,
                        class_names=class_names,
                        # transform=weights.transforms(), # optionally pass in a specified transform from our pretrained model weights
                        image_size=(224, 224))

Woohoo!

Those predictions look far better than the ones our TinyVGG model was previously making.

### 6.1 Making predictions on a custom image

It looks like our model does well qualitatively on data from the test set.

But how about on our own custom image?

That's where the real fun of machine learning is!

Predicting on your own custom data, outisde of any training or test set.

To test our model on a custom image, let's import the old faithful `pizza-dad.jpeg` image (an image of my dad eating pizza).

We'll then pass it to the `pred_and_plot_image()` function we created above and see what happens.

In [ ]:
# Download custom image
import requests

# Setup custom image path
custom_image_path = data_path / "04-pizza-dad.jpeg"



# Predict on custom image
pred_and_plot_image(model=model,
                    image_path=custom_image_path,
                    class_names=class_names)

Two thumbs up!

Looks like our model go it right again!

But this time the prediction probability is higher than the one from TinyVGG (`0.373`) in [04. PyTorch Custom Datasets section 11.3](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function).

This indicates our `efficientnet_b0` model is *more* confident in its prediction where as our TinyVGG model was par with just guessing.

## Main takeaways
* **Transfer learning** often allows to you get good results with a relatively small amount of custom data.
* Knowing the power of transfer learning, it's a good idea to ask at the start of every problem, "does an existing well-performing model exist for my problem?"
* When using a pretrained model, it's important that your custom data be formatted/preprocessed in the same way that the original model was trained on, otherwise you may get degraded performance.
* The same goes for predicting on custom data, ensure your custom data is in the same format as the data your model was trained on.
* There are [several different places to find pretrained models](https://www.learnpytorch.io/06_pytorch_transfer_learning/#where-to-find-pretrained-models) from the PyTorch domain libraries, HuggingFace Hub and libraries such as `timm` (PyTorch Image Models).

## Exercises

All of the exercises are focused on practicing the code above.

You should be able to complete them by referencing each section or by following the resource(s) linked.

All exercises should be completed using [device-agnostic code](https://pytorch.org/docs/stable/notes/cuda.html#device-agnostic-code).

**Resources:**
* [Exercise template notebook for 06](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/exercises/06_pytorch_transfer_learning_exercises.ipynb)
* [Example solutions notebook for 06](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/06_pytorch_transfer_learning_exercise_solutions.ipynb) (try the exercises *before* looking at this)
    * See a live [video walkthrough of the solutions on YouTube](https://youtu.be/ueLolShyFqs) (errors and all)

1. Make predictions on the entire test dataset and plot a confusion matrix for the results of our model compared to the truth labels. Check out [03. PyTorch Computer Vision section 10](https://www.learnpytorch.io/03_pytorch_computer_vision/#10-making-a-confusion-matrix-for-further-prediction-evaluation) for ideas.
2. Get the "most wrong" of the predictions on the test dataset and plot the 5 "most wrong" images. You can do this by:
    * Predicting across all of the test dataset, storing the labels and predicted probabilities.
    * Sort the predictions by *wrong prediction* and then *descending predicted probabilities*, this will give you the wrong predictions with the *highest* prediction probabilities, in other words, the "most wrong".
    * Plot the top 5 "most wrong" images, why do you think the model got these wrong?
3. Predict on your own image of pizza/steak/sushi - how does the model go? What happens if you predict on an image that isn't pizza/steak/sushi?
4. Train the model from section 4 above for longer (10 epochs should do), what happens to the performance?
5. Train the model from section 4 above with more data, say 20% of the images from Food101 of Pizza, Steak and Sushi images.
    * You can find the [20% Pizza, Steak, Sushi dataset](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/data/pizza_steak_sushi_20_percent.zip) on the course GitHub. It was created with the notebook [`extras/04_custom_data_creation.ipynb`](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/04_custom_data_creation.ipynb).
6. Try a different model from [`torchvision.models`](https://pytorch.org/vision/stable/models.html) on the Pizza, Steak, Sushi data, how does this model perform?
    * You'll have to change the size of the classifier layer to suit our problem.
    * You may want to try an EfficientNet with a higher number than our B0, perhaps `torchvision.models.efficientnet_b2()`?
  
## Extra-curriculum
* Look up what "model fine-tuning" is and spend 30-minutes researching different methods to perform it with PyTorch. How would we change our code to fine-tine? Tip: fine-tuning usually works best if you have *lots* of custom data, where as, feature extraction is typically better if you have less custom data.
* Check out the new/upcoming [PyTorch multi-weights API](https://pytorch.org/blog/introducing-torchvision-new-multi-weight-support-api/) (still in beta at time of writing, May 2022), it's a new way to perform transfer learning in PyTorch. What changes to our code would need to made to use the new API?
* Try to create your own classifier on two classes of images, for example, you could collect 10 photos of your dog and your friends dog and train a model to classify the two dogs. This would be a good way to practice creating a dataset as well as building a model on that dataset.